In [3]:
import arcpy
from arcgis.gis import *
#these next two support random string generation
from random import choice
from string import ascii_uppercase


arcpy.env.overwriteOutput = True

#Get the layer from the TOC
featurelayer = "ZoneTemplate"
zoneNameField = "displocname"
fields = ["displocname","routename","objectid","shape@"]

#Set an expression to optionally use as a whereclause
expression = "{} = 'DEN'".format(arcpy.AddFieldDelimiters(featurelayer, zoneNameField))

#create a table in scratch space to use as output 
explicit_scratch = "C:/temp"
GDB_GUID = ''.join(choice(ascii_uppercase) for i in range(12))
explicit_scratch_ws = arcpy.CreateFileGDB_management(explicit_scratch, "scratch_" + GDB_GUID + ".gdb")
arcpy.env.workspace = os.path.join(explicit_scratch, "scratch_" + GDB_GUID + ".gdb")

failedZoneCheck_tbl = arcpy.management.CreateTable(arcpy.env.workspace, "failedZoneCheck")
arcpy.AddField_management(failedZoneCheck_tbl, "slivers", "TEXT", field_alias="Sliver Polygons", field_length = 5)
arcpy.AddField_management(failedZoneCheck_tbl, "toodense", "TEXT", field_alias="Excessive Vertices", field_length = 5)
arcpy.AddField_management(failedZoneCheck_tbl, "donuts", "TEXT", field_alias="Interior Voids", field_length = 5)
arcpy.AddField_management(failedZoneCheck_tbl, "manyparts", "TEXT", field_alias="Exessive Parts", field_length = 5)
arcpy.AddField_management(failedZoneCheck_tbl, "sourceOID", "LONG", field_alias="Source OBJECTID")
arcpy.AddField_management(failedZoneCheck_tbl, "zoneinfo", "TEXT", field_alias="Zone Metadata", field_length = 200)

#Set the fields we will use for the insert cursor
insertFields = ['slivers','toodense','donuts','manyparts','sourceOID','zoneinfo']
updateCur = arcpy.da.InsertCursor(failedZoneCheck_tbl,insertFields)

#We are sorting the zone records by service area name and zone name, which will order them so that duplicate geometry zones are 
#grouped together. We will use a "lastArea" shape.area tracker to recognize when we have encountered a new shape to evaluate - no need to report 
#issues with each identical shape record. 
lastArea = int(0)
print ("Slivers, Excessive Vertices, Interior Voids, Excessive Parts, OID, Shape Specifics") 

#uncomment the version with a whereclause if you want to evaluate a subset of zones - or just set a definition query on the layer
for row in sorted(arcpy.da.SearchCursor(featurelayer, fields)):
#for row in sorted(arcpy.da.SearchCursor(featurelayer, fields, where_clause = expression)):
    shape = row[3]
    thisArea = int(shape.area)
    if (thisArea != lastArea):
        if(shape.length > shape.area):
            sliverShape = True
        else:
            sliverShape = False
        if(shape.length/shape.pointCount <200):
            excessiveVertices = True
        else:
            excessiveVertices = False
        if(shape.boundary().partCount - 1 > shape.partCount):
            interiorRings = True
        else:
            interiorRings = False
        if(shape.partCount > 4):
            manyParts = True
        else:
            manyParts = False
        if(sliverShape or excessiveVertices or interiorRings or manyParts): 
            rowMetadata = "DispLoc={} RouteName={} parts={:d} length={:d} area={:d} point count={:d} ring count={:d}".format(
                                                                                                      row[0],
                                                                                                      row[1],
                                                                                                      shape.partCount, 
                                                                                                      int(shape.length), 
                                                                                                      int(shape.area), shape.pointCount, 
                                                                                                      shape.boundary().partCount)
            
            updateCur.insertRow((str(sliverShape),str(excessiveVertices),str(interiorRings),str(manyParts),row[2],rowMetadata))
            print(str(sliverShape) +  "," + str(excessiveVertices) +  "," + str(interiorRings) +  "," +  str(manyParts) +  "," + rowMetadata)
        
                   
        lastArea = thisArea
del updateCur          

Slivers, Excessive Vertices, Interior Voids, Excessive Parts, OID, Shape Specifics
False,False,True,False,DispLoc=ABC RouteName=ABCN parts=2 length=1094062 area=9966623390 point count=1159 ring count=58
False,False,True,False,DispLoc=ABC RouteName=ABCW parts=1 length=989373 area=10664017204 point count=950 ring count=35
False,False,True,False,DispLoc=ABC RouteName=ABCE parts=2 length=734292 area=6943725857 point count=748 ring count=28
False,False,True,False,DispLoc=ABC RouteName=ABC_LG parts=2 length=636477 area=5623342121 point count=759 ring count=29
False,False,True,False,DispLoc=ABC RouteName=ABC_City parts=1 length=316778 area=1699683040 point count=462 ring count=23
False,False,True,False,DispLoc=ABC RouteName=ABC_Gated parts=1 length=890466 area=8401019328 point count=1117 ring count=39
False,False,True,False,DispLoc=ABC RouteName=ABC_Resi parts=2 length=325441 area=1532937368 point count=457 ring count=39
